In [1]:
import sqlite3
import json

In [2]:
def extract_order_info_from_email(email_info):
    # Connect to the SQLite database
    conn = sqlite3.connect('C:/Users/grzegorz.kosek/Desktop/My Projects/Data & AI/GenAI/GenAI%20Traning/sql_db/gen_ai.db')
    cursor = conn.cursor()

    try:
        # Extract information from the provided email
        email_content, email_date, client_address = email_info

        # Query to retrieve client ID based on the client's address from the email
        cursor.execute("SELECT client_id FROM clients WHERE address = ?", (client_address,))
        client_id = cursor.fetchone()

        if client_id:
            client_id = client_id[0]

            # Query to fetch previous orders for the identified client
            cursor.execute("""
                SELECT client_orders.order_no, products.product_name, client_orders.order_date, client_orders.quantity, client_orders.payment_status
                FROM client_orders
                INNER JOIN products ON client_orders.product_id = products.product_id
                WHERE client_orders.client_id = ?
                ORDER BY client_orders.order_date DESC
                """, (client_id,))

            previous_orders = cursor.fetchall()

            if previous_orders:
                # Convert fetched orders into JSON format
                orders_info = []
                for order in previous_orders:
                    order_info = {
                        'order_no': order[0],
                        'product_name': order[1],
                        'order_date': order[2],
                        'quantity': order[3],
                        'payment_status': order[4]
                    }
                    orders_info.append(order_info)

                # Convert orders info to JSON and print it
                orders_json = json.dumps(orders_info, indent=4)
                print(orders_json)
            else:
                print("No previous orders found for the client associated with the provided address.")

        else:
            print("No client found with the provided address.")

    except sqlite3.Error as e:
        print("Error querying the database:", e)

    finally:
        # Close the database connection
        conn.close()

# Example email information
email_info = (
    'Hello, I hope this email finds you well. I am writing to inquire about the status of my recent order for Headphones B&W. The order number is #1, and it was placed on October 31, 2023. Could you please provide me with an update on the shipment? Thank you, Arek Pierwszy Pawia 1, Poland',
    '2023-11-13 08:00:00',
    'Pawia 1'
)

email_info_2 = (
    'Dear [Sales Team], I hope this email reaches you in good health. I wanted to confirm the shipment of my order for Speakers B&W (Order #4), placed on October 31, 2023. Can you provide tracking details for the delivery? Thanks, Darek Treci Pawia 3, Poland',
    '2023-11-13 18:00:00',
    'Pawia 3'
)

# Extract order information based on the provided email
extract_order_info_from_email(email_info_2)

[
    {
        "order_no": 7,
        "product_name": "Amp Sony",
        "order_date": "2023-10-31",
        "quantity": 2,
        "payment_status": "Paid"
    },
    {
        "order_no": 8,
        "product_name": "Amp B&O",
        "order_date": "2023-10-30",
        "quantity": 1,
        "payment_status": "Pending"
    },
    {
        "order_no": 9,
        "product_name": "Amp B&W",
        "order_date": "2023-10-29",
        "quantity": 3,
        "payment_status": "Pending"
    },
    {
        "order_no": 10,
        "product_name": "Cables",
        "order_date": "2023-10-28",
        "quantity": 2,
        "payment_status": "Paid"
    }
]
